In [1]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [2]:
import versor as vsr
from versor.drawing import *

In [3]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
n_points = 10
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.3, 3)).null() 
            for i in range(n_points)]
points_b = [point.spin(motor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]




In [4]:
points_a = [vsr.Vec(0,0,0).null(), 
            vsr.Vec(1,0,0).null(), 
            vsr.Vec(0,1,0).null(), 
            vsr.Vec(0,0,1).null(), 
            vsr.Vec(0,0,-1).null(),
            vsr.Vec(-1,0,0).null(),
            vsr.Vec(0,-1,0).null()]

In [46]:
scene = Scene(children=
              [point_mesh(points_a[0], color=Colors.GREEN)] +\
              [point_mesh(p, color=Colors.HOTPINK) for p in points_a[1:]] +\
              [plane_mesh(points_a[0]- b, color=Colors.LIGHTSKYBLUE, width=1, height=1) for b in points_a[1:]] +\
              [frame_mesh(), AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, |
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

SyntaxError: invalid syntax (<ipython-input-46-c8e36d9aba97>, line 9)

In [47]:
a = points_a[0]
out = np.sum([np.outer(b ^ a, b ^ a) for b in points_a[1:]],0)
w,v = np.linalg.eig(out)
sw = np.sort(w)

In [42]:
w

array([ 0. ,  0. ,  0. ,  2. ,  2. ,  2. ,  0. ,  0. ,  0. ,  1.5])

In [44]:
Q = vsr.Par(*w).dual()
Q.radius()

0.4330127018922193

In [45]:
Q.pnt()

Pnt: [ -0.25 -0.25 -0.25 1 0.09375 ]

In [39]:
par = b ^ a
par.__print_debug_info_console()

In [8]:
c_lin = (sw[2] - sw[1]) / np.sum(sw)
c_pln = 2 * (sw[1] - sw[0]) / np.sum(sw)
c_sph = 3 * sw[0] / np.sum(sw)

In [9]:
print(c_lin, c_pln, c_sph)

0.0 0.0 0.0


In [10]:
np.linalg.eigh?

In [11]:

np.linalg.eig?

In [12]:
def new_positions_spherical_coordinates(num_points, radius, x_offset=1.0, y_offset=2.0, z_offset=3.0, stddev=0.0):
    radius = radius + stddev * np.random.rand()
    theta = np.random.uniform(0.,2. * np.pi ,(num_points,1))
    phi = np.arccos(1-2*np.random.uniform(0.0,1.,(num_points,1)))
    x = radius * np.sin( theta ) * np.cos( phi ) + x_offset + stddev * np.random.rand()
    y = radius * np.sin( theta ) * np.sin( phi ) + y_offset + stddev * np.random.rand()
    z = radius * np.cos( theta ) + z_offset + stddev * np.random.rand()
    return (x,y,z)

In [13]:
x,y,z = new_positions_spherical_coordinates(10, 4)

In [14]:
ps = [vsr.Vec(x,y,z).null() for x,y,z in zip(x,y,z) ]

In [15]:
ps = points_a
M = np.zeros((5,5))
M[:3,:3] = np.eye(3)
M[3,4] = -1
M[4,3] = -1
print(M)
P = np.zeros((5,5)) 
for p in ps:
    P += np.outer(p,p)
PM = np.dot(P,M)
(w,v) = np.linalg.eig(PM)
idx = np.where(w == np.min(w[w>0]))[0][0]
x_ = v[:,idx]
x_ /= x_[3]
x_dorst = x_.copy()

print(x_)
radius = np.sqrt(x_[0]*x_[0] + x_[1]*x_[1] + x_[2]*x_[2] - 2*x_[4])
print("Estimated radius: {}".format(radius))

[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0. -1.]
 [ 0.  0.  0. -1.  0.]]
[-0.     -0.     -0.      1.     -0.4629]
Estimated radius: 0.962195458196


In [16]:
P

array([[ 2. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  2. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  2. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  7. ,  3. ],
       [ 0. ,  0. ,  0. ,  3. ,  1.5]])

In [17]:
scene = Scene(children=
              [point_mesh(p, color=Colors.HOTPINK) for p in ps] +\
              [sphere_mesh(vsr.Pnt(*x_dorst).undual(), color=Colors.LIGHTSKYBLUE)] +\
              [frame_mesh(), AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)